In [2]:
import pandas as pd

In [3]:

# Read all CSV files
ranking = pd.read_csv("ranking.csv", parse_dates=["STANDINGSDATE"])
teams = pd.read_csv("teams.csv")
players = pd.read_csv("players.csv")
players_data = pd.read_csv("players_data.csv")
games = pd.read_csv("games.csv", parse_dates=["GAME_DATE_EST"])
game_details = pd.read_csv("games_details.csv")


C:\Users\pc\AppData\Local\Temp\ipykernel_17492\180698479.py:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  game_details = pd.read_csv("games_details.csv")


In [4]:
# Step 1: Prepare dim_season (combine seasons from all datasets)
# From ranking.csv (SEASON_ID like 22022 -> 2022)
ranking['season_type'] = ranking['SEASON_ID'].astype(str).str[0]
ranking = ranking[ranking['season_type'] == '2'].copy()
ranking['season_year'] = ranking['SEASON_ID'].astype(str).str[1:].astype(int)

# From players.csv (SEASON like 2019)
players['season_year'] = players['SEASON'].astype(int)

# From players_data.csv (season like 1996-97)
players_data['season_year'] = players_data['season'].apply(lambda x: int(x.split('-')[0]))

# From games.csv (SEASON like 2022)
games['season_year'] = games['SEASON'].astype(int)

# Combine unique season years from all datasets
all_season_years = pd.concat([
    pd.Series(ranking['season_year'].unique()),
    pd.Series(players['season_year'].unique()),
    pd.Series(players_data['season_year'].unique()),
    pd.Series(games['season_year'].unique())
]).unique()
dim_season = pd.DataFrame(all_season_years, columns=['season_year'])
dim_season['season_id'] = dim_season.index + 1

In [5]:
# Map season_year to season_id in all datasets
ranking = ranking.merge(dim_season[['season_year', 'season_id']], on='season_year', how='left')
players = players.merge(dim_season[['season_year', 'season_id']], on='season_year', how='left')
players_data = players_data.merge(dim_season[['season_year', 'season_id']], on='season_year', how='left')
games = games.merge(dim_season[['season_year', 'season_id']], on='season_year', how='left')

In [6]:
# Step 2: Prepare dim_team from teams.csv
dim_team = teams[['TEAM_ID', 'ABBREVIATION', 'NICKNAME', 'YEARFOUNDED', 'MIN_YEAR', 'MAX_YEAR', 'CITY', 'ARENA', 'ARENACAPACITY', 'OWNER', 'GENERALMANAGER', 'HEADCOACH']].copy()
dim_team.rename(columns={
    'TEAM_ID': 'team_id',
    'ABBREVIATION': 'abbreviation',
    'NICKNAME': 'nickname',
    'YEARFOUNDED': 'year_founded',
    'MIN_YEAR': 'start_year',
    'MAX_YEAR': 'end_year',
    'CITY': 'city',
    'ARENA': 'arena',
    'ARENACAPACITY': 'arena_capacity',
    'OWNER': 'owner',
    'GENERALMANAGER': 'general_manager',
    'HEADCOACH': 'head_coach'
}, inplace=True)

# Ensure team_id is an integer
dim_team['team_id'] = dim_team['team_id'].astype(int)

# Ensure year_founded, start_year, end_year are integers
dim_team['year_founded'] = dim_team['year_founded'].astype(int)
dim_team['start_year'] = dim_team['start_year'].astype(int)
dim_team['end_year'] = dim_team['end_year'].astype(int)

# Fill NaN in arena_capacity with 0 and ensure it's an integer
dim_team['arena_capacity'] = dim_team['arena_capacity'].fillna(0).astype(int)

# Fill NaN in string columns with 'Unknown'
dim_team.fillna({
    'abbreviation': 'Unknown',
    'nickname': 'Unknown',
    'city': 'Unknown',
    'arena': 'Unknown',
    'owner': 'Unknown',
    'general_manager': 'Unknown',
    'head_coach': 'Unknown'
}, inplace=True)

# Add conference from ranking.csv
team_conference = ranking[['TEAM_ID', 'CONFERENCE']].drop_duplicates().copy()
team_conference.rename(columns={'TEAM_ID': 'team_id', 'CONFERENCE': 'conference'}, inplace=True)
team_conference = (team_conference.groupby('team_id')['conference']
                   .agg(lambda x: x.mode()[0] if not x.empty else 'Unknown')
                   .reset_index())
dim_team = dim_team.merge(team_conference, on='team_id', how='left')
dim_team['conference'] = dim_team['conference'].fillna('Unknown')

# Ensure team_id is unique
dim_team = dim_team.drop_duplicates(subset=['team_id'])


In [7]:
# Step 3: Prepare dim_ranking from ranking.csv
# Drop unnecessary columns
ranking.drop(columns=['RETURNTOPLAY', 'LEAGUE_ID', 'TEAM', 'W_PCT', 'HOME_RECORD', 'ROAD_RECORD'], inplace=True)

# Get the last entry for each TEAM_ID per season_year
latest_per_season = (ranking.sort_values(["TEAM_ID", "season_year", "STANDINGSDATE"])
                     .groupby(["TEAM_ID", "season_year"])
                     .last()
                     .reset_index())
# Step 4: Prepare dim_date (combine dates from ranking.csv and games.csv)
# Dates from ranking.csv
ranking_dates = pd.DataFrame(latest_per_season['STANDINGSDATE'].unique(), columns=['date'])

# Dates from games.csv
game_dates = pd.DataFrame(games['GAME_DATE_EST'].unique(), columns=['date'])

# Combine unique dates
all_dates = pd.concat([ranking_dates, game_dates]).drop_duplicates().reset_index(drop=True)
dim_date = pd.DataFrame(all_dates, columns=['date'])
dim_date['date'] = pd.to_datetime(dim_date['date'])
dim_date['year'] = dim_date['date'].dt.year.astype(int)
dim_date['month'] = dim_date['date'].dt.month.astype(int)
dim_date['day'] = dim_date['date'].dt.day.astype(int)
dim_date['date_id'] = dim_date.index + 1

# Map STANDINGSDATE to date_id
latest_per_season = latest_per_season.merge(dim_date[['date', 'date_id']], 
                                          left_on='STANDINGSDATE', 
                                          right_on='date', 
                                          how='left')

# Prepare dim_ranking with date_id
dim_ranking = latest_per_season[['TEAM_ID', 'season_id', 'CONFERENCE', 'G', 'W', 'L', 'date_id']].copy()
dim_ranking.rename(columns={
    'TEAM_ID': 'team_id',
    'CONFERENCE': 'conference',
    'G': 'game_played',
    'W': 'wins',
    'L': 'lose'
}, inplace=True)
dim_ranking['ranking_id'] = dim_ranking.index + 1

# Ensure numeric columns are integers
dim_ranking['team_id'] = dim_ranking['team_id'].astype(int)
dim_ranking['season_id'] = dim_ranking['season_id'].astype(int)
dim_ranking['game_played'] = dim_ranking['game_played'].astype(int)
dim_ranking['wins'] = dim_ranking['wins'].astype(int)
dim_ranking['lose'] = dim_ranking['lose'].astype(int)
dim_ranking['date_id'] = dim_ranking['date_id'].astype(int)
dim_ranking['ranking_id'] = dim_ranking['ranking_id'].astype(int)

# Fill any missing string columns
dim_ranking['conference'] = dim_ranking['conference'].fillna('Unknown')

# Drop temporary columns
dim_ranking.drop(columns=['SEASON_ID', 'season_year', 'season_type'], inplace=True, errors='ignore')


In [ ]:
# Step 5: Prepare dim_player_static and dim_player_dynamic
# Prepare dim_player_static
dim_player_static = players[['PLAYER_ID', 'PLAYER_NAME']].drop_duplicates().copy()
dim_player_static.rename(columns={'PLAYER_ID': 'player_id', 'PLAYER_NAME': 'player_name'}, inplace=True)

# Ensure player_id is an integer
dim_player_static['player_id'] = dim_player_static['player_id'].astype(int)

# Merge with players_data to get static attributes
players_data_static = players_data[['player_name', 'player_height', 'college', 'country', 'draft_year', 'draft_round', 'draft_number']].drop_duplicates(subset=['player_name']).copy()
dim_player_static = dim_player_static.merge(players_data_static, on='player_name', how='left')

# Rename player_height to height and keep it as a numeric value (in cm)
dim_player_static.rename(columns={'player_height': 'height'}, inplace=True)

# Fill missing static attributes with defaults
dim_player_static.fillna({
    'draft_year': 'Undrafted',
    'draft_round': 'Undrafted',
    'draft_number': 'Undrafted',
    'height': 0,
    'college': 'Unknown',
    'country': 'Unknown'
}, inplace=True)


# Ensure player_id is unique
dim_player_static = dim_player_static.drop_duplicates(subset=['player_id'])

# Prepare dim_player_dynamic
# From players.csv (no weight data)
dim_player_dynamic_players = players[['PLAYER_ID', 'TEAM_ID', 'season_id']].copy()
dim_player_dynamic_players.rename(columns={'PLAYER_ID': 'player_id', 'TEAM_ID': 'team_id'}, inplace=True)
dim_player_dynamic_players['weight'] = None
dim_player_dynamic_players['source'] = 'players'

# Ensure player_id and team_id are integers
dim_player_dynamic_players['player_id'] = dim_player_dynamic_players['player_id'].astype(int)
dim_player_dynamic_players['team_id'] = dim_player_dynamic_players['team_id'].astype(int)
dim_player_dynamic_players['season_id'] = dim_player_dynamic_players['season_id'].astype(int)

# From players_data.csv (has weight data)
dim_player_dynamic_data = players_data[['player_name', 'team_abbreviation', 'season_id', 'player_weight']].copy()
dim_player_dynamic_data.rename(columns={'player_weight': 'weight'}, inplace=True)

# Ensure weight is numeric
dim_player_dynamic_data['weight'] = pd.to_numeric(dim_player_dynamic_data['weight'], errors='coerce')

# Map player_name to player_id
dim_player_dynamic_data = dim_player_dynamic_data.merge(dim_player_static[['player_name', 'player_id']], on='player_name', how='left')

# Drop rows with NaN player_id (unmatched players)
dim_player_dynamic_data = dim_player_dynamic_data.dropna(subset=['player_id'])

# Ensure player_id is an integer after merge
dim_player_dynamic_data['player_id'] = dim_player_dynamic_data['player_id'].astype(int)

# Map team_abbreviation to team_id using dim_team
dim_player_dynamic_data = dim_player_dynamic_data.merge(dim_team[['abbreviation', 'team_id']], 
                                                        left_on='team_abbreviation', 
                                                        right_on='abbreviation', 
                                                        how='left')

# Fill NaN team_id with 0 (indicating unknown team)
dim_player_dynamic_data['team_id'] = dim_player_dynamic_data['team_id'].fillna(0).astype(int)

# Ensure season_id is an integer
dim_player_dynamic_data['season_id'] = dim_player_dynamic_data['season_id'].astype(int)

# Select columns for dim_player_dynamic
dim_player_dynamic_data = dim_player_dynamic_data[['player_id', 'team_id', 'season_id', 'weight']]
dim_player_dynamic_data['source'] = 'players_data'

# Combine both sources
if not dim_player_dynamic_data.empty:
    dim_player_dynamic = pd.concat([dim_player_dynamic_players, dim_player_dynamic_data], ignore_index=True)
else:
    print("\nWarning: dim_player_dynamic_data is empty after dropping unmatched players.")
    dim_player_dynamic = dim_player_dynamic_players.copy()

# Ensure player_id, team_id, and season_id are integers after concatenation
dim_player_dynamic['player_id'] = dim_player_dynamic['player_id'].astype(int)
dim_player_dynamic['team_id'] = dim_player_dynamic['team_id'].astype(int)
dim_player_dynamic['season_id'] = dim_player_dynamic['season_id'].astype(int)

# Clean dim_player_dynamic
dim_player_dynamic = dim_player_dynamic.drop_duplicates(subset=['player_id', 'team_id', 'season_id'])

# Fill missing weights with 0
dim_player_dynamic['weight'] = dim_player_dynamic['weight'].fillna(0)

# Drop the source column
dim_player_dynamic.drop(columns=['source'], inplace=True)

C:\Users\pc\AppData\Local\Temp\ipykernel_17492\3877749635.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_player_dynamic = pd.concat([dim_player_dynamic_players, dim_player_dynamic_data], ignore_index=True)


In [16]:
# Step 6: Prepare fact_game from games.csv
fact_game = games[['GAME_ID', 'GAME_DATE_EST', 'season_id', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'PTS_home', 'PTS_away', 'HOME_TEAM_WINS']].copy()
fact_game.rename(columns={
    'GAME_ID': 'game_id',
    'HOME_TEAM_ID': 'home_team_id',
    'VISITOR_TEAM_ID': 'visitor_team_id',
    'PTS_home': 'home_points',
    'PTS_away': 'visitor_points',
    'HOME_TEAM_WINS': 'home_team_wins'
}, inplace=True)

print(fact_game.isna().sum())
print(fact_game.dtypes)

# Map GAME_DATE_EST to date_id
fact_game = fact_game.merge(dim_date[['date', 'date_id']], 
                            left_on='GAME_DATE_EST', 
                            right_on='date', 
                            how='left')

# Drop the date column
fact_game.drop(columns=['GAME_DATE_EST', 'date'], inplace=True)

# Ensure numeric columns are integers
# fact_game['game_id'] = fact_game['game_id'].astype(int)
# fact_game['season_id'] = fact_game['season_id'].astype(int)
# fact_game['home_team_id'] = fact_game['home_team_id'].astype(int)
# fact_game['visitor_team_id'] = fact_game['visitor_team_id'].astype(int)
# fact_game['home_points'] = fact_game['home_points'].astype(int)
# fact_game['visitor_points'] = fact_game['visitor_points'].astype(int)
# fact_game['date_id'] = fact_game['date_id'].astype(int)

# Ensure home_team_wins is boolean
fact_game['home_team_wins'] = fact_game['home_team_wins'].astype(bool)

# Step 7: Prepare dim_player_performance from game_details.csv
dim_player_performance = game_details[['GAME_ID', 'PLAYER_ID', 'TEAM_ID', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS']].copy()
dim_player_performance.rename(columns={
    'GAME_ID': 'game_id',
    'PLAYER_ID': 'player_id',
    'TEAM_ID': 'team_id',
    'MIN': 'minutes_played',
    'FGM': 'field_goals_made',
    'FGA': 'field_goals_attempted',
    'FG_PCT': 'field_goal_percentage',
    'FG3M': 'three_pointers_made',
    'FG3A': 'three_pointers_attempted',
    'FG3_PCT': 'three_pointer_percentage',
    'FTM': 'free_throws_made',
    'FTA': 'free_throws_attempted',
    'FT_PCT': 'free_throw_percentage',
    'OREB': 'offensive_rebounds',
    'DREB': 'defensive_rebounds',
    'REB': 'total_rebounds',
    'AST': 'assists',
    'STL': 'steals',
    'BLK': 'blocks',
    'TO': 'turnovers',
    'PF': 'personal_fouls',
    'PTS': 'points',
}, inplace=True)

# Generate performance_id
dim_player_performance['performance_id'] = dim_player_performance.index + 1

# Ensure numeric columns are integers where appropriate
# dim_player_performance['game_id'] = dim_player_performance['game_id'].astype(int)
# dim_player_performance['player_id'] = dim_player_performance['player_id'].astype(int)
# dim_player_performance['team_id'] = dim_player_performance['team_id'].astype(int)
# dim_player_performance['field_goals_made'] = dim_player_performance['field_goals_made'].fillna(0).astype(int)
# dim_player_performance['field_goals_attempted'] = dim_player_performance['field_goals_attempted'].fillna(0).astype(int)
# dim_player_performance['three_pointers_made'] = dim_player_performance['three_pointers_made'].fillna(0).astype(int)
# dim_player_performance['three_pointers_attempted'] = dim_player_performance['three_pointers_attempted'].fillna(0).astype(int)
# dim_player_performance['free_throws_made'] = dim_player_performance['free_throws_made'].fillna(0).astype(int)
# dim_player_performance['free_throws_attempted'] = dim_player_performance['free_throws_attempted'].fillna(0).astype(int)
# dim_player_performance['offensive_rebounds'] = dim_player_performance['offensive_rebounds'].fillna(0).astype(int)
# dim_player_performance['defensive_rebounds'] = dim_player_performance['defensive_rebounds'].fillna(0).astype(int)
# dim_player_performance['total_rebounds'] = dim_player_performance['total_rebounds'].fillna(0).astype(int)
# dim_player_performance['assists'] = dim_player_performance['assists'].fillna(0).astype(int)
# dim_player_performance['steals'] = dim_player_performance['steals'].fillna(0).astype(int)
# dim_player_performance['blocks'] = dim_player_performance['blocks'].fillna(0).astype(int)
# dim_player_performance['turnovers'] = dim_player_performance['turnovers'].fillna(0).astype(int)
# dim_player_performance['personal_fouls'] = dim_player_performance['personal_fouls'].fillna(0).astype(int)
# dim_player_performance['points'] = dim_player_performance['points'].fillna(0).astype(int)
# dim_player_performance['plus_minus'] = dim_player_performance['plus_minus'].fillna(0).astype(int)
# dim_player_performance['performance_id'] = dim_player_performance['performance_id'].astype(int)

# # Fill missing percentages with 0
# dim_player_performance['field_goal_percentage'] = dim_player_performance['field_goal_percentage'].fillna(0)
# dim_player_performance['three_pointer_percentage'] = dim_player_performance['three_pointer_percentage'].fillna(0)
# dim_player_performance['free_throw_percentage'] = dim_player_performance['free_throw_percentage'].fillna(0)

# Fill missing minutes_played with '0:00'
# dim_player_performance['minutes_played'] = dim_player_performance['minutes_played'].fillna('0:00')

# Ensure only valid player_id and team_id (foreign key constraints)
dim_player_performance = dim_player_performance[dim_player_performance['player_id'].isin(dim_player_static['player_id'])]
dim_player_performance = dim_player_performance[dim_player_performance['team_id'].isin(dim_team['team_id'])]


game_id             0
GAME_DATE_EST       0
season_id           0
home_team_id        0
visitor_team_id     0
home_points        99
visitor_points     99
home_team_wins      0
dtype: int64
game_id                     int64
GAME_DATE_EST      datetime64[ns]
season_id                   int64
home_team_id                int64
visitor_team_id             int64
home_points               float64
visitor_points            float64
home_team_wins              int64
dtype: object


In [14]:
fact_game

,game_id,season_id,home_team_id,visitor_team_id,home_points,visitor_points,home_team_wins,date_id
0,22200477,1,1610612740,1610612759,126.0,117.0,True,21
1,22200478,1,1610612762,1610612764,120.0,112.0,True,21
2,22200466,1,1610612739,1610612749,114.0,106.0,True,22
3,22200467,1,1610612755,1610612765,113.0,93.0,True,22
4,22200468,1,1610612737,1610612741,108.0,110.0,False,22
...,...,...,...,...,...,...,...,...
26646,11400007,21,1610612737,1610612740,93.0,87.0,True,4322
26647,11400004,21,1610612741,1610612764,81.0,85.0,False,4322
26648,11400005,21,1610612747,1610612743,98.0,95.0,True,4322
26649,11400002,21,1610612761,1610612758,99.0,94.0,True,4323
